In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Step 1: Importing Libraries

In [ ]:
 # Common Libraries
 import numpy as np
 import cv2
 from matplotlib import pyplot as plt

 #ML Libraries

 #DL Libraries
 import tensorflow as tf
 from tensorflow import keras
 from tensorflow.keras.utils import to_categorical
 from tensorflow.keras.models import Sequential
 from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
 from tensorflow.keras.preprocessing.image import ImageDataGenerator
 from tensorflow.keras.datasets import cifar100

# Step 2: Load the Dataset / Read The DataSet

In [ ]:
# load the sata set and split it between train and test sets
(x_train, y_train),(x_test, y_test) = cifar100.load_data()

In [ ]:
type(x_train), x_train.shape, y_train.shape, x_train.ndim, x_train.dtype 

# Step 3: Data Processing / Data Preperation

In [ ]:
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (32, 32, 3)

In [ ]:
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

In [ ]:
num_classes = 100

In [ ]:
# convert class vectors to binary class matrics
y_train = keras.utils.to_categorical(y_train,
                                     num_classes)
y_test = keras.utils.to_categorical(y_test,
                                    num_classes)
y_test.shape, y_train.shape # Sparse Label

In [ ]:
# reshape dataset to have a single channel
trainX=x_train.reshape((x_train.shape[0], 32, 32, 3))
testX=x_test.reshape((x_test.shape[0], 32, 32, 3))

In [ ]:
X_train,X_test =  trainX,testX

# Step 4:Build the model

In [ ]:
from tensorflow.keras.layers import BatchNormalization

In [ ]:
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
model = keras.Sequential()
# Depthwise
model.add(DepthwiseConv2D(kernel_size=3,
                          activation="relu", padding ="same",  input_shape=(32,32,3)))
#Pointwise
model.add(Conv2D(32,kernel_size=(1, 1),
                      activation="relu", padding ="same"))
# 3X1 --> Intermediate Output --> 1X3 --> Final Output
model.add(BatchNormalization())
model.add(DepthwiseConv2D(kernel_size=3,
                      activation="relu",padding ="same"))
model.add(Conv2D(32,kernel_size=(1, 1),
                      activation="relu", padding ="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(DepthwiseConv2D(kernel_size=3,
                      activation="relu", padding ="same"))
model.add(Conv2D(64,kernel_size=(1, 1),
                      activation="relu", padding ="same"))
model.add(BatchNormalization())
model.add(DepthwiseConv2D(kernel_size=3,
                      activation="relu",padding ="same"))
model.add(Conv2D(64,kernel_size=(1, 1),
                      activation="relu", padding ="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(DepthwiseConv2D(kernel_size=3,
                      activation="relu", padding ="same"))
model.add(Conv2D(128,kernel_size=(1, 1),
                      activation="relu", padding ="same"))
model.add(BatchNormalization())
model.add(DepthwiseConv2D(kernel_size=3,
                      activation="relu",padding ="same"))
model.add(Conv2D(128,kernel_size=(1, 1),
                      activation="relu", padding ="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(DepthwiseConv2D(kernel_size=3,
                      activation="relu", padding ="same"))
model.add(Conv2D(256,kernel_size=(1, 1),
                      activation="relu", padding ="same"))
model.add(BatchNormalization())
model.add(DepthwiseConv2D(kernel_size=3,
                      activation="relu",padding ="same"))
model.add(Conv2D(256,kernel_size=(1, 1),
                      activation="relu", padding ="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024,activation="relu" ))
model.add(Dropout(0.25))
model.add(Dense(512,activation="relu" ))  
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation="softmax")) 
model.summary() 

In [ ]:
from tensorflow.keras.optimizers import Adagrad
opt = Adagrad(learning_rate=0.001,weight_decay=0.002, ema_momentum=0.9)
model.compile(optimizer=opt,
loss='categorical_crossentropy',
               metrics=['accuracy'])

In [ ]:
batch_size = 128
epochs = 5
history = model.fit(X_train, y_train,
  batch_size=batch_size,
  epochs=epochs,
  verbose=1,
  validation_data=(X_test, y_test))

In [ ]:
# evaluate the model
_, acc = model.evaluate(X_test, y_test, verbose=0)
print('> %.3f' % (acc * 100.0))

In [ ]:
scores, histories = list(), list()
# stores scores
scores.append(acc)
histories.append(history)

In [ ]:
# print summary
print('Accuracy: mean=%.3f std=%.3f, n=%d'
      % (np.mean(scores)*100,
      np.std(scores)*100, len(scores)))
# box and whisker plots of results
plt.boxplot(scores)
plt.show()

In [ ]:
for i in range(len(histories)):
# plot loss
 plt.subplot(2, 1, 1)
 plt.title('Cross Entropy Loss')
 plt.plot(histories[i].history['loss'],
          color='blue', label='train')
 plt.plot(histories[i].history['val_loss'],
          color='orange', label='test')
# plot accuracy
 plt.subplot(2, 1, 2)
 plt.title('Classification Accuracy')
 plt.plot(histories[i].history['accuracy'],
          color='blue', label='train')
 plt.plot(histories[i].history['val_accuracy'],
          color='orange', label='test')
 plt.show()